In [ ]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# 1.Loading Data
df = pd.read_csv('train.csv').sample(frac=0.5, random_state=42)
print(f"Dataset loaded successfully. Rows: {len(df)}")

df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

df['hour'] = df['pickup_datetime'].dt.hour
df['day_of_week'] = df['pickup_datetime'].dt.dayofweek  
df['month'] = df['pickup_datetime'].dt.month

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)

    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

df['distance_km'] = haversine_distance(
    df['pickup_latitude'], df['pickup_longitude'],
    df['dropoff_latitude'], df['dropoff_longitude']
)

#2.Data Cleaning

df = df[(df['trip_duration'] < 10800) & (df['trip_duration'] > 60)]
df = df[(df['distance_km'] > 0.1) & (df['distance_km'] < 100)]

print(f"Data after cleaning: {len(df)} rows")

# 3. Model Training
features = ['pickup_longitude', 'pickup_latitude',
            'dropoff_longitude', 'dropoff_latitude',
            'distance_km', 'hour', 'day_of_week']

X = df[features]
y = df['trip_duration']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("\nTraining Random Forest Model")

rf_model = RandomForestRegressor(n_estimators=50, min_samples_leaf=10, random_state=42, n_jobs=-1)

rf_model.fit(X_train, y_train)

# 4. Evaluation
print("\nEvaluating Model Performance...")
y_pred = rf_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Success Metrics:")
print(f"RMSE (Root Mean Squared Error): {rmse:.2f} seconds (~{rmse/60:.1f} minutes)")
print(f"R2 Score: {r2:.3f}")

# 5. Saving the Model
model_filename = 'nyc_taxi_model.pkl'
joblib.dump(rf_model, 'nyc_taxi_model.pkl', compress=3)

print(f"\nModel saved as '{model_filename}'")

Dataset loaded successfully. Rows: 729322
Data after cleaning: 720087 rows

Training Random Forest Model

Evaluating Model Performance...
Success Metrics:
RMSE (Root Mean Squared Error): 304.13 seconds (~5.1 minutes)
R2 Score: 0.789

Model saved as 'nyc_taxi_model.pkl'
